# Neural Network

## 딥러닝 맛보기

- 추후 사용할 RNN을 위하여 딥러닝 즉 신경망에 대해 맛보기하는 시간을 가저보자
- 이전 다른 분류기도 그랬지만 분류와 회귀 모두에서 사용할 수 있다.
- 수학적 원리는 정확히 설명하고 구현하기 어려우므로 다른 자료를 참조하기를 권장함
- 기본 구조 외에 다양한 architecture에 대한 것은 [여기](https://github.com/young31/Deep-Learning/tree/master/Architecture)
  
    
- keras를 사용하여 구현할 예정
    - keras는 쉽게 사용하게 해주는 툴
    - tensorflow 등을 백으로 가질 수 있음
    
- 순차모형과 이름을 달아 연결하는 방법이 있음
    - 순차모형이 시작하기는 쉬움
    - 나중에 복잡한 망을 짜려면 개별 망에 연결하는 방법을 사용
    
- 신경망은 greedy하게 해를 찾는 경사하강법을 사용해서 답을 찾아가므로 정규화를 무조건 해줘야함!!!
    - 해당 절에서는 편의를 위해 카테고리변수를 따로 설정하지 않음

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = pd.read_csv('../1.clustering/titanic.csv')

target = data['Survived']
data = data.drop(['PassengerId', 'Survived'], axis=1)

def sex(a):
    if a == 'male':
        return 0
    else:
        return 1
    
def emb(a):
    if a == 'S':
        return 0
    elif a == 'Q':
        return 1
    else:
        return 2
    
data['Sex'] = data['Sex'].map(sex)
data['Embarked'] = data['Embarked'].map(emb)

train_X, test_X, train_y, test_y = train_test_split(data, target, test_size=0.3, random_state=2019, shuffle=True)

In [2]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
tt = std.fit_transform(train_X)
te = std.fit_transform(test_X)

In [3]:
## 층을 만든다
from keras import models, layers, optimizers
import keras.backend as K
K.clear_session()

input_layer = layers.Input(shape=train_X.shape[1:])

layer1 = layers.Dense(32, activation='relu')(input_layer)
layer2 = layers.Dense(16, activation='relu')(layer1)

out_layer = layers.Dense(1, activation='sigmoid')(layer2)

model = models.Model(input_layer, out_layer)

Using TensorFlow backend.


In [4]:
## compile 한다.

model.compile(optimizer = optimizers.Adam(lr=0.1),
             loss='binary_crossentropy',
             metrics=['acc'])

In [5]:
## callback 함수를 쓴다면 추가
from keras.callbacks import EarlyStopping, ModelCheckpoint

cp = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True)
es = EarlyStopping(monitor='val_acc', patience=10, restore_best_weights=True, verbose=1)

In [6]:
## 훈련시킨다.
model.fit(tt, train_y,
         epochs=40,
         validation_split=0.3,
         callbacks=[cp, es])


Train on 348 samples, validate on 150 samples
Epoch 1/40
348/348 [==============================] - 1s 2ms/step - loss: 0.6948 - acc: 0.7241 - val_loss: 0.4506 - val_acc: 0.8267
Epoch 2/40
348/348 [==============================] - 0s 143us/step - loss: 0.4975 - acc: 0.7730 - val_loss: 0.5426 - val_acc: 0.7600
Epoch 3/40
348/348 [==============================] - 0s 201us/step - loss: 0.5055 - acc: 0.8103 - val_loss: 0.4855 - val_acc: 0.7800
Epoch 4/40
348/348 [==============================] - 0s 163us/step - loss: 0.4590 - acc: 0.7787 - val_loss: 0.5110 - val_acc: 0.8067
Epoch 5/40
348/348 [==============================] - 0s 186us/step - loss: 0.4269 - acc: 0.8190 - val_loss: 0.4496 - val_acc: 0.8067
Epoch 6/40
348/348 [==============================] - 0s 193us/step - loss: 0.4110 - acc: 0.8103 - val_loss: 0.4342 - val_acc: 0.8200
Epoch 7/40
348/348 [==============================] - 0s 192us/step - loss: 0.3921 - acc: 0.8103 - val_loss: 0.4173 - val_acc: 0.8333
Epoch 8/40
348/348

In [7]:
## 모델 평가
res = model.predict(te)
def thres(a):
    if a < 0.5:
        return 0
    else:
        return 1
res = list(map(thres, res))
print(accuracy_score(test_y, res))

0.8271028037383178


In [8]:
model.evaluate(te, test_y)

214/214 [==============================] - 0s 84us/step


[0.47494250452407055, 0.827102780342102]

In [9]:
## 모형을 쌓듯이 하는 방법의 같은 코드
model = models.Sequential()

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))

model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer = optimizers.Adam(lr=0.1),
             loss='binary_crossentropy',
             metrics=['acc'])

model.fit(tt, train_y.values,
         epochs=40,
         validation_split=0.3,
         callbacks=[cp, es])


Train on 348 samples, validate on 150 samples
Epoch 1/40
348/348 [==============================] - 0s 516us/step - loss: 0.5818 - acc: 0.7126 - val_loss: 0.4872 - val_acc: 0.7533
Epoch 2/40
348/348 [==============================] - 0s 189us/step - loss: 0.5967 - acc: 0.7213 - val_loss: 0.4958 - val_acc: 0.7600
Epoch 3/40
348/348 [==============================] - 0s 206us/step - loss: 0.4994 - acc: 0.7241 - val_loss: 0.4719 - val_acc: 0.7667
Epoch 4/40
348/348 [==============================] - 0s 206us/step - loss: 0.4858 - acc: 0.8103 - val_loss: 0.4905 - val_acc: 0.7800
Epoch 5/40
348/348 [==============================] - 0s 149us/step - loss: 0.4388 - acc: 0.8333 - val_loss: 0.4988 - val_acc: 0.7867
Epoch 6/40
348/348 [==============================] - 0s 152us/step - loss: 0.4366 - acc: 0.8132 - val_loss: 0.4805 - val_acc: 0.7467
Epoch 7/40
348/348 [==============================] - 0s 232us/step - loss: 0.4176 - acc: 0.8420 - val_loss: 0.5312 - val_acc: 0.7600
Epoch 8/40
348/3